In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np


In [6]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

NameError: name 'requests' is not defined

In [2]:

pages = set()  
def getLinks_in(pageUrl):
                table=pd.DataFrame()
                html = urlopen('https://uk.wikipedia.org{}'.format(pageUrl))
                bs = BeautifulSoup(html, 'html.parser')
                try:
                    print(bs.h1.get_text())
                    print(bs.find(id='mw-pages').find_all('p')[0])
                except AttributeError:
                    print('This page is missing something! Continuing.')
                for link in bs.find(id='mw-pages').find_all('a', href=re.compile('^(/wiki/)')):
                    if 'href' in link.attrs:
                        if link.attrs['href'] not in pages:
                          # We have encountered a new page
                            newPage = link.attrs['href']
                            #print('-' * 20)
                            html = urlopen('https://uk.wikipedia.org{}'.format(newPage))
                            bs = BeautifulSoup(html, 'html.parser')
                            try:
                                city=bs.h1.text
                                print('city  ' + bs.h1.text)
                            except AttributeError:
                                city = None
                                #print('Error city')

                            # year
                            try:
                                year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Заснован(е|о)')).find_parent('tr').find_all('td')[1].text.strip()
                                if  not year or year=='' or year=='невідомо' or year=='не відомо' or year=='—':
                                    #print ('i try')
                                    try:
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[1].text.strip()
                                        #print('2 year  ' + year)
                                    except AttributeError:
                                        try:
                                            year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                            #print('3 year  ' + year)
                                        except AttributeError:
                                            year = None  
                                    except IndexError:
                                        try:
                                            year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                            #print('3 year  ' + year)
                                        except AttributeError:
                                            year = None        
                                else:   
                                    year
                                    #print('4 year  ' + year)


                            except IndexError:
                                #print('index error  ' + year)
                                try: 
                                    if not year or year=='' or year=='невідомо'  or year=='не відомо' or year=='—':
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Заснован(е|о)')).find_parent('tr').find_all('td')[0].text.strip()
                                        #print('5 year  ' + year)
                                except AttributeError:
                                    year = None
                                except IndexError:
                                    if not year :
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                        #print('6 year  ' + year)
                            except AttributeError:
                                try:
                                    year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[1].text.strip()
                                    #print('7 year  ' + year)

                                except AttributeError:
                                    year = None        




                            # population
                            try:
                                population = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Населення')).find_parent('tr').find_all('td')[1].find(text=True, recursive=False).strip()
                                #print('population  ' + population)
                            except IndexError:
                                try:
                                    population = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Населення')).find_parent('tr').find_all('td')[0].find(text=True, recursive=False).strip()
                                    #print('population  ' + population)
                                except AttributeError:
                                    population = None
                            except AttributeError:
                                population = None
                                #print('Error population')

                            # latitude
                            try:
                                latitude = bs.find('span', {'class': 'latitude'}).text
                                #print('latitude  ' + latitude)
                            except AttributeError:
                                try: 
                                    latitude = bs.find('a', {'class': 'mw-kartographer-maplink'}).text
                                    #latitude=latitude[:17]
                                    if latitude.find('″') == -1:
                                        latitude=latitude[:13]
                                    else: 
                                        latitude=latitude[:17]
                                except AttributeError:
                                    latitude = None


                            # longitude
                            try:
                                longitude = bs.find('span', {'class': 'longitude'}).text
                                #print('longitude 0 ' + longitude)
                            except AttributeError:
                                try:
                                    longitude = bs.find('a', {'class': 'mw-kartographer-maplink'}).text
                                    if longitude.find('″') == -1:
                                        longitude=longitude[13:]
                                    else: 
                                        longitude=longitude[17:]
                                except AttributeError:
                                    longitude = None

                            data= pd.DataFrame ({
                                "city": [city],
                                "year": [year],
                                "population": [population],
                                "latitude": [latitude],
                                "longitude": [longitude]

                            })
                            table=table.append(data,ignore_index=True)
                            pages.add(newPage)


                return table
def getLinks(pageUrl):
    global pages
    global table

    table=pd.DataFrame()
    html = urlopen('https://uk.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')                      
    for link in bs.find(id='mw-pages').find_all('a', href=re.compile('^(/w/index.php)'),text="наступна сторінка"):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # We have encountered a new page
                newPage_main = link.attrs['href']
                print('------*' * 10)
   
               
        #getLinks_in(newPage_main)
        table=table.append(getLinks_in(newPage_main),ignore_index=True)
        #print('----step1-----')
        #print(table)
        #getLinks(newPage_main)
        table=table.append(getLinks(newPage_main),ignore_index=True)
        #print('----step2-----')
        #print(table)
            
    return table 


In [3]:
#міста
#links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%9C%D1%96%D1%81%D1%82%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')
#смт
#links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A1%D0%B5%D0%BB%D0%B8%D1%89%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')
#села
links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A1%D0%B5%D0%BB%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')

In [4]:


table_pandas1=getLinks_in(links)
table_pandas2=getLinks(links)
#print('--------final-1--------')
#print(table_pandas1)
#print('--------final-2--------')
#print(table_pandas2)
#print('--------final-3--------')
table_pandas=table_pandas1.append(table_pandas2,ignore_index=True)
print(table_pandas)


Категорія:Села України
<p>Показано 200 сторінок цієї категорії (із 27 399).
</p>
city  Села України
city  Абазівка (Красноградський район)
city  Абазівка (Полтавський район)
city  Абрамів Яр (Куп'янський район)
city  Абрамівка (Вишгородський район)
city  Абрамівка (Полтавський район)
city  Абрамівська Долина
city  Абрамок
city  Абранка
city  Абрикосівка (Кам'янець-Подільський район)
city  Абрикосівка (Кіровський район)
city  Абрикосівка (Михайлівський район)
city  Абрикосівка (Сакський район)
city  Абрикосове (Білгород-Дністровський район)
city  Абрикосове (Первомайський район)
city  Авангард (Веселівський район)
city  Августинівка
city  Августівка (Одеський район)
city  Августівка (Прилуцький район)
city  Августівка (Тернопільський район)
city  Авдіївка (Куликівський район)
city  Авдіївка (Сосницький район)


KeyboardInterrupt: 

In [8]:
table_pandas.tail()

,city,year,population,latitude,longitude
1169,Ясне (Джанкойський район),None,102,45°46′25″ пн. ш.,34°16′09″ сх. д.
1170,Ясне (Докучаєвська міська рада),None,769 (01.01.2017),47°46′32″ пн. ш.,37°38′06″ сх. д.
1171,Ясне (Лутугинський район),None,339,48°23′57″ пн. ш.,39°05′10″ сх. д.
1172,Ясне (Чернівецький район),1500,20,48°35′25″ пн. ш.,28°09′34″ сх. д.
1173,Яснодольськ,1988,288,48°35′10″ пн. ш.,38°46′14″ сх. д.


In [9]:
table_pandas.head()

,city,year,population,latitude,longitude
0,Абрикосівка (селище),1928,1204,46°29′17″ пн. ш.,33°04′16″ сх. д.
1,Авангард (Кам'янський район),None,91 (,48°26′56″ пн. ш.,33°41′12″ сх. д.
2,Авангард (Криворізька міська рада),None,1522,48°00′35″ пн. ш.,33°18′58″ сх. д.
3,Адамівка (Уманський район),до 1926,55,49°08′53″ пн. ш.,29°56′30″ сх. д.
4,Азов (селище),1823,907,47°27′19″ пн. ш.,37°06′34″ сх. д.


In [10]:
#міста
#table_pandas.to_csv(r'C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_city.csv', index = False, header = True)
#смт
table_pandas.to_csv(r'C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_SMT.csv', index = False, header = True)
#села
#table_pandas.to_csv(r'C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_vilige.csv', index = False, header = True)


In [11]:
table_pandas

,city,year,population,latitude,longitude
0,Абрикосівка (селище),1928,1204,46°29′17″ пн. ш.,33°04′16″ сх. д.
1,Авангард (Кам'янський район),None,91 (,48°26′56″ пн. ш.,33°41′12″ сх. д.
2,Авангард (Криворізька міська рада),None,1522,48°00′35″ пн. ш.,33°18′58″ сх. д.
3,Адамівка (Уманський район),до 1926,55,49°08′53″ пн. ш.,29°56′30″ сх. д.
4,Азов (селище),1823,907,47°27′19″ пн. ш.,37°06′34″ сх. д.
...,...,...,...,...,...
1169,Ясне (Джанкойський район),None,102,45°46′25″ пн. ш.,34°16′09″ сх. д.
1170,Ясне (Докучаєвська міська рада),None,769 (01.01.2017),47°46′32″ пн. ш.,37°38′06″ сх. д.
1171,Ясне (Лутугинський район),None,339,48°23′57″ пн. ш.,39°05′10″ сх. д.
1172,Ясне (Чернівецький район),1500,20,48°35′25″ пн. ш.,28°09′34″ сх. д.
